# Using Metadata to Improve Artifical Intelligance Medical Image Diagnostic Accuracy
**Purpose and Background**
Conduct a descriptive analysis of crowdsourced data extracted from user interaction with a mobile application where tasked to binarly (yes or no) identify abnormalities in medical images. 

Two user categories were differentiated: Medical experts hired to interact with the application; and crowd, anyone who downloaded and used the application.

**Show that the crowd agrees with the expert majority more than experts agreeing with the expert majority**


### Import datasets

In [29]:
import pandas as pd
import numpy as np
results = pd.read_csv('1345_customer_results.csv') #medical case results
admin = pd.read_csv('1345_admin_reads.csv') #raw individual read

### Inspect Customer Results

In [30]:
results.dtypes

Case ID                   int64
Origin                   object
Origin Created At        object
Content ID                int64
URL                      object
Labeling State           object
Series                  float64
Series Index            float64
Patch                   float64
Qualified Reads           int64
Correct Label            object
Majority Label           object
Difficulty              float64
Agreement               float64
First Choice Answer      object
First Choice Votes        int64
First Choice Weight     float64
Second Choice Answer     object
Second Choice Votes       int64
Second Choice Weight    float64
Internal Notes          float64
Comments                 object
Explanation             float64
dtype: object

**Preliminary filtering for security purposes**


In [31]:
results = results.dropna(subset=['Origin']) 
results["Expert Reads"] = results["Origin"].str.extract(r'vote(\d)').astype(float)
results = results.drop(['Origin Created At','Origin','Content ID','URL'],axis=1)

In [32]:
results.head(2)

,Case ID,Labeling State,Series,Series Index,Patch,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Internal Notes,Comments,Explanation,Expert Reads
0,5888087,Gold Standard,NaN,NaN,NaN,2,'no','no',0.0,1.0,'no',2,1.54,'yes',0,0.0,NaN,[],NaN,2.0
1,5888088,Gold Standard,NaN,NaN,NaN,3,'no','no',0.0,1.0,'no',3,2.34,'yes',0,0.0,NaN,[],NaN,0.0


**Inspect NaN Columns for Content**

In [24]:
results.loc[results['Series'].notna()| results['Series Index'].notna() | results['Patch'].notna() | results['Internal Notes'].notna() | results['Explanation'].notna()]

,Case ID,Labeling State,Series,Series Index,Patch,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Internal Notes,Comments,Explanation,Expert Votes


Dataframe is empty; None of the columns scanned through the pipeline contained any data

In [25]:
results = results.drop(['Series','Series Index','Patch','Internal Notes','Explanation'],axis=1)

**Inspect Comments for Relevance**

In [26]:
results.loc[results['Comments'] != '[]']

,Case ID,Labeling State,Qualified Reads,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Comments,Expert Votes
4245,5892332,Gold Standard,1,'no','no',0.000,1.000,'no',1,0.80,'yes',0,0.00,['There was rapid and spiky rates so why am I ...,3.0
6029,5894116,Gold Standard,5,'no','yes',1.000,1.000,'yes',5,4.00,'no',0,0.00,['Can someone explain why the answer is “no”?'],0.0
8346,5896433,Gold Standard,3,'yes','no',1.000,1.000,'no',3,2.32,'yes',0,0.00,['??'],5.0
11433,5899520,Gold Standard,2,'yes','no',1.000,1.000,'no',2,1.58,'yes',0,0.00,"[""i can't see any spike in this question so wh...",5.0
12911,5900998,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.56,'no',0,0.00,['There is obviously a peak happened in there'],3.0
13827,5901914,Gold Standard,6,'yes','no',1.000,1.000,'no',6,4.72,'yes',0,0.00,['No spike present'],5.0
13953,5902040,Gold Standard,2,'no','yes',1.000,1.000,'yes',2,1.58,'no',0,0.00,['How?'],3.0
16033,5904120,Gold Standard,1,'yes','no',1.000,1.000,'no',1,0.78,'yes',0,0.00,['How? '],6.0
16326,5904413,Gold Standard,3,'yes','no',1.000,1.000,'no',3,2.46,'yes',0,0.00,['Multiple?'],6.0
16385,5904472,Gold Standard,3,'yes','yes',0.333,0.667,'yes',2,1.58,'no',1,0.78,[' Wtf'],5.0


None seem relevant, so the Comments column will be dropped

In [27]:
results = results.drop(['Comments'],axis=1)

In [ ]:
results = results[results["Qualified Reads"] != 0] 
results = results[results["Qualified Reads"] >= 9] 

#### Important columns for analysis
Each row corresponds to a medical image case 

**Identifiers:** 

Case ID: unique identifier will serve as index

Labeling State: identifies whether a expert consensus has been achieved (yes=Gold Standard, no= In Progress)

URL: Extracted out expert vote count within the URL 

**Reads and Annotations**

Qualified Reads: crowd vote count

Expert Reads: expert vote count

Correct Label: expert consensus {yes=case is abnormal, no=case is normal, NaN=no consensus}

Majority Label: crowd consensus 

**Measures of Confidence**

Difficulty: Qualified Reads *without the Correct Label* divided by total Qualified Reads.

Agreement: Qualified Reads *with the Majority Label* divided by total Qualified Reads.

Nth Choice Answer: crowd answer (First Choice is the Majority Label)
        
Nth Choice Votes: number of crowd votes
        
Nth Choice Weight:
        
        
        



### Add Additional Relevant Columns 

In [66]:
results["Split"] = results["Qualified Reads"] == 4
split_exp_opinion = results['Split']==True
print(results['Split'])
print(split_exp_opinion)

0        False
1        False
2        False
3        False
4        False
         ...  
30288    False
30289    False
30290    False
30291    False
30292    False
Name: Split, Length: 30293, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
30288    False
30289    False
30290    False
30291    False
30292    False
Name: Split, Length: 30293, dtype: bool


In [68]:
results

,Case ID,Crowd Votes,Correct Label,Majority Label,Difficulty,Agreement,First Choice Answer,First Choice Votes,First Choice Weight,Second Choice Answer,Second Choice Votes,Second Choice Weight,Expert Votes,Split,Total votes
77,5888164,10,'no','no',0.0,1.000,'no',10,7.90,'yes',0,0.00,0.0,False,10.0
85,5888172,9,'no','no',0.0,1.000,'no',9,6.95,'yes',0,0.00,0.0,False,9.0
87,5888174,9,'no','no',0.0,1.000,'no',9,7.08,'yes',0,0.00,0.0,False,9.0
96,5888183,9,'no','no',0.0,1.000,'no',9,7.17,'yes',0,0.00,NaN,False,NaN
115,5888202,9,'no','no',0.0,1.000,'no',9,7.17,'yes',0,0.00,0.0,False,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30257,5918344,11,NaN,'yes',NaN,0.909,'yes',10,7.88,'no',1,0.90,4.0,False,15.0
30268,5918355,10,'yes','yes',0.2,0.800,'yes',8,6.47,'no',2,1.64,7.0,False,17.0
30271,5918358,13,NaN,'yes',NaN,0.769,'yes',10,7.84,'no',3,2.36,4.0,False,17.0
30272,5918359,10,'yes','yes',0.0,1.000,'yes',10,8.06,'no',0,0.00,5.0,False,15.0


what was the intention and what did I get out of running this code.

In [69]:
print(results["Difficulty"].describe())

count    1415.000000
mean        0.126514
std         0.217765
min         0.000000
25%         0.000000
50%         0.000000
75%         0.111000
max         1.000000
Name: Difficulty, dtype: float64
